In [201]:
from nltk.tokenize import word_tokenize
from urllib.request import urlretrieve, urlopen
from bs4 import BeautifulSoup
import csv
import re, string

### Get a Complete List of Movies We Have Ratings For

In [4]:
fileloc = "/Volumes/Eriks HD/MLC Final/ml-latest/"

In [7]:
movie_file = open(fileloc + "movies.csv").readlines()

In [38]:
lines = []
for line in csv.reader(open(fileloc + "movies.csv", newline=''), delimiter=',', quotechar='\"'):
    lines.append(line)

In [86]:
def split_title(title):
    title = title.strip()
    return [title[0:-7], int(title[-5:-1])]

In [88]:
movies = {}
for i in range(1,len(lines)):
    try:
        line = lines[i]
        idnum = int(line[0])
        title, year = split_title(line[1])
        genres = line[2].split('|')

        movies[idnum] = [title, year, genres]
        
    except Exception as inst:
        print(lines[i])
        
    

['40697', 'Babylon 5', 'Sci-Fi']
['79607', 'Millions Game, The (Das Millionenspiel)', 'Action|Drama|Sci-Fi|Thriller']
['87442', 'Bicycle, Spoon, Apple (Bicicleta, cullera, poma)', 'Documentary']
['98063', 'Mona and the Time of Burning Love (Mona ja palavan rakkauden aika) (1983))', 'Drama']
['107434', 'Diplomatic Immunity (2009– )', 'Comedy']
['108548', 'Big Bang Theory, The (2007-)', 'Comedy']
['112406', 'Brazil: In the Shadow of the Stadiums', 'Documentary']
['113190', 'Slaying the Badger', 'Documentary']
['115133', 'Tatort: Im Schmerz geboren', 'Crime']
['115685', 'National Theatre Live: Frankenstein', 'Drama|Fantasy']
['125571', 'The Court-Martial of Jackie Robinson', '(no genres listed)']
['125632', 'In Our Garden', '(no genres listed)']
['125958', 'Stephen Fry In America - New World', '(no genres listed)']
['126438', 'Two: The Story of Roman & Nyro', 'Documentary|Drama']
['126929', "Li'l Quinquin", '(no genres listed)']
['127005', 'A Year Along the Abandoned Road', '(no genres li

In [89]:
for i in range(1,30,5):
    print(movies[i])

['Toy Story', 1995, ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']]
['Heat', 1995, ['Action', 'Crime', 'Thriller']]
['American President, The', 1995, ['Comedy', 'Drama', 'Romance']]
['Casino', 1995, ['Crime', 'Drama']]
['Get Shorty', 1995, ['Comedy', 'Crime', 'Thriller']]
['Othello', 1995, ['Drama']]


In [90]:
len(movies)

30075

### Query Rotten Tomatoes for Each Movie

Not using their public API, but hopefully we'll only have to do this once. I don't think people take kindly to scrapers.

In [186]:
rt_url = "http://www.rottentomatoes.com"
rt_search_url = rt_url + "/search/?search="

In [218]:
exclude = set(string.punctuation)

def get_search_url(movie):
    """
    Finds the rotten tomatoes URL for the given title.
    """
    title = ''.join(ch for ch in movie[0] if ch not in exclude)
    search_url = rt_search_url + '+'.join(title.lower().split(' '))
    
    return search_url

for i in range(8,100):
    print(get_search_url(movies[i]))

http://www.rottentomatoes.com/search/?search=tom+and+huck
http://www.rottentomatoes.com/search/?search=sudden+death
http://www.rottentomatoes.com/search/?search=goldeneye
http://www.rottentomatoes.com/search/?search=american+president+the
http://www.rottentomatoes.com/search/?search=dracula+dead+and+loving+it
http://www.rottentomatoes.com/search/?search=balto
http://www.rottentomatoes.com/search/?search=nixon
http://www.rottentomatoes.com/search/?search=cutthroat+island
http://www.rottentomatoes.com/search/?search=casino
http://www.rottentomatoes.com/search/?search=sense+and+sensibility
http://www.rottentomatoes.com/search/?search=four+rooms
http://www.rottentomatoes.com/search/?search=ace+ventura+when+nature+calls
http://www.rottentomatoes.com/search/?search=money+train
http://www.rottentomatoes.com/search/?search=get+shorty
http://www.rottentomatoes.com/search/?search=copycat
http://www.rottentomatoes.com/search/?search=assassins
http://www.rottentomatoes.com/search/?search=powder
ht

KeyError: 91

In [191]:
def process_url(movie):
    """
    Given a movie, determines the rotten tomatoes URL for the critics' reviews of that movie.
    """
    search_url = get_search_url(movie)
    
    response = urlopen(search_url)
    soup = BeautifulSoup(response)
    
    if "search results" in soup.find("title").text.lower():
        url = rt_url + soup.find(id="movie_results_ul").findChildren()[0].find('a')['href']
    
    else:
        url = response.url.split('?')[0]

    return url + "reviews/"

In [195]:
"""
Please don't reset this!
"""
errored_movies = []

In [221]:
for i in range(1,100):
    """
    Note: I'm doing this in intervals, and updating every now and then. Currently done with
    1 - 100.
    """
    if i not in movies:
        continue
        
    try:
        url = process_url(movies[i])
    except:
        errored_movies.append(movies[i])
        continue
        
    if (len(movies[i]) <= 3):
        movies[i].append(url)

In [225]:
for i in range(1,100):
    if i not in movies:
        continue
    if len(movies[i]) > 3:
        print(movies[i][3])

http://www.rottentomatoes.com/m/toy_story/reviews/
http://www.rottentomatoes.com/m/1068044-jumanji/reviews/
http://www.rottentomatoes.com/m/grumpier_old_men/reviews/
http://www.rottentomatoes.com/m/waiting_to_exhale/reviews/
http://www.rottentomatoes.com/m/father_of_the_bride_part_ii/reviews/
http://www.rottentomatoes.com/m/1068182-heat/reviews/
http://www.rottentomatoes.com/m/1018047-sabrina/reviews/
http://www.rottentomatoes.com/m/tom_and_huck/reviews/
http://www.rottentomatoes.com/m/1068470-sudden_death/reviews/
http://www.rottentomatoes.com/m/goldeneye/reviews/
http://www.rottentomatoes.com/m/american_president/reviews/
http://www.rottentomatoes.com/m/dracula_dead_and_loving_it/reviews/
http://www.rottentomatoes.com/m/balto/reviews/
http://www.rottentomatoes.com/m/nixon/reviews/
http://www.rottentomatoes.com/m/cutthroat_island/reviews/
http://www.rottentomatoes.com/m/1067987-casino/reviews/
http://www.rottentomatoes.com/m/1068832-sense_and_sensibility/reviews/
http://www.rottentoma

In [ ]:
errored_movies